In [2]:
# Modelo com random forest

import pandas as pd
import spacy
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import language_tool_python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# Carregar modelo do spaCy
nlp = spacy.load('en_core_web_sm')

# Instanciar o LanguageTool
tool = language_tool_python.LanguageTool('en-US')

# Funções para extração de características
def count_grammar_errors(text):
    matches = tool.check(text)
    return len(matches)

def calculate_lexical_diversity(doc):
    return len(set([token.text.lower() for token in doc])) / len(doc)

# Classe para extrair características
class TextFeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        features = []
        for doc in self.nlp.pipe(X):
            features.append([
                len(doc),  # Comprimento do texto
                count_grammar_errors(doc.text),  # Número de erros gramaticais
                calculate_lexical_diversity(doc)  # Diversidade lexical
            ])
        return np.array(features)

# Carregar o DataFrame
df = pd.read_csv('train.csv')
df['full_text'] = df['full_text'].str.lower().str.replace('\n', ' ')
n_sample = 120

# Função para amostragem
def sample(df, group_column, n):
    return df.groupby(group_column).apply(lambda x: x.sample(n=min(len(x), n))).reset_index(drop=True)

# Filtrar e amostrar dados
filter = df[df['score'].isin([1, 2, 3, 4, 5, 6])]
df_sample = sample(filter, 'score', n_sample)

# Instanciar o extrator de características
extractor = TextFeaturesExtractor(nlp)

# Aplicar a transformação no DataFrame
features_array = extractor.transform(df_sample['full_text'])

# Criar novas colunas no DataFrame com as características extraídas
df_sample['comprimento_texto'] = features_array[:, 0]
df_sample['num_grammatical_errors'] = features_array[:, 1]
df_sample['lexical_diversity'] = features_array[:, 2]

# Calcular a correlação entre as variáveis
correlation_matrix = df_sample[['score', 'comprimento_texto', 'num_grammatical_errors', 'lexical_diversity']].corr()
correlation_matrix

# Preparar os dados para o modelo de classificação
X = df_sample[['comprimento_texto', 'num_grammatical_errors', 'lexical_diversity']]
y = df_sample['score']

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instanciar e treinar o classificador
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Acurácia: {accuracy}')
print('Relatório de Classificação:')
print(report)

# Função para prever a nota de um novo texto
def predict_score(text):
    doc = nlp(text.lower().replace('\n', ' '))
    features = np.array([[len(doc), count_grammar_errors(doc.text), calculate_lexical_diversity(doc)]])
    score_pred = clf.predict(features)
    return score_pred[0]

# Função para prever notas de vários textos em um DataFrame
def predict_scores(df, text_column):
    features = extractor.transform(df[text_column])
    scores_pred = clf.predict(features)
    df['predicted_score'] = scores_pred
    return df


Acurácia: 0.4791666666666667
Relatório de Classificação:
              precision    recall  f1-score   support

           1       0.58      0.50      0.54        28
           2       0.36      0.32      0.34        28
           3       0.34      0.38      0.36        26
           4       0.42      0.36      0.39        22
           5       0.48      0.62      0.54        21
           6       0.75      0.79      0.77        19

    accuracy                           0.48       144
   macro avg       0.49      0.50      0.49       144
weighted avg       0.48      0.48      0.48       144



In [3]:
# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')

Acurácia: 0.4791666666666667


In [8]:
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score

scores = cross_val_score(clf, X_test, y_test, cv=5, scoring='f1_macro')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.42 (+/- 0.30)
